In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools # finding permutations of articles
import heapq
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
users_interactions = pd.read_csv("../input/users_interactions.csv")
users_interactions.head(30)

In [ ]:
byUserId = users_interactions.sort_values('personId').loc[users_interactions['eventType'] == 'VIEW'].groupby(by='personId')
#steps = users_interactions['contentId'].unique()
steps = byUserId['contentId'].unique()
steps = pd.DataFrame(index=steps,columns=['support','highest-prob'])
user = byUserId.get_group(1908339160857512799).sort_values('timestamp',ascending=False)
print(user['contentId'].value_counts())
print(user[['timestamp','contentId']])

For each possible k-order state and each given article, we want to find the support of the that state, and the support of that state with that article after it for a given user. 
We then divide the two to get the probability of that step coming after that state. We'll build an order 3 Markov model, which means the biggest state it will consider will be of size 3.

In [ ]:
class MarkovTree:
    #data structure of single node, with id of content (step) and list of content that appears directly before this step in the data.
    #alpha is the Laplacian smoothing hyper-parameter, sigma is the cardinality of all the steps. 
    #next is dictionary of most probable next steps from given state and their counts
    #IMPLEMENT Laplacian smoothing 
    
    #Add n x n transition matrix
    def __init__(self, id,alpha,sigma):
        self.id = id
        #needs to be heap based on confidence?
        self.children = []
        self.support = 1
        self.alpha = alpha
        self.sigma = sigma
        self.next = Counter({})
        self.prob = 0
    def add(self,id):
        #adds state to children. If this is A and id is B, this method denotes there exists sequence BA in the sequence of events
        if id not in self.children:
            child = MarkovTree(id,self.alpha,self.sigma,)
            self.children.append(child)
            return self.children[-1]
        else:
            index = self.children.index(id)
            self.children[index].support += 1
            return self.children[index]
            
    def subSearch(self,id):
        #searches for id in children of this branch and returns relevant support
        
        
        ids = [n.id for n in self.children]
        if id in ids:
            index = ids.index(id)
            return self.children[index]
        else:
            return None
        
    def search(self,state):
        #searches tree for sequence of steps 
        #could write logic better
        current = self
        for i,item in state.iteritems():
            print(item)
            nxt = current.subSearch(item)
            
            if nxt == None:
                return current
            current = nxt
        return current
            
    def getSupport(self):
        #get support of current state
        return self.support
    
    def addNext(self,id):
        #Adds state to list of probable next steps for current state
        if (id in self.next.keys()):
            self.next[id] += 1
        else:
            self.next[id] = 1
    
    def getNext(self):
        #Returns most probable next step. If not enough information, returns most commonly visited site
        if any(self.next):
            return sorted(next,key=next.__getitem__,reverse=True)[0]
        else:
            #might be wrong syntax
            return sorted(self.children,key=getSupport(),reverse=True)[0]
        
class MTMarkovModel:
    def __init__(self, user,k):
        self.user = user
        self.k = k
        self.alpha = 0.25
        #whats a good value for alpha?
        self.tree = MarkovTree(-1,self.alpha,len(user))
        self.max = user['contentId'].value_counts().iloc[0]
        self.mean = user['contentId'].value_counts().mean()
        #self.steps = steps
    def buildModel(self):
        #Add children to self.tree for each state that appears in the user's history
        for i in range(1,len(self.user['contentId'])):
            self.tree.add(self.user['contentId'].iloc[i]).addNext(self.user['contentId'].iloc[i-1])
            #Set next
            #print(self.tree.children[-1])
            
        #Add children for higher order states
        for i in range(1,self.k):
            for j in range(1,len(self.user['contentId'])-(i)):
                
                dest = self.tree.search(self.user['contentId'].iloc[j:j+i]) 
                #print(self.user['contentId'].iloc[j+1:j+i+1],dest)
                if (dest):
                    dest.add(self.user['contentId'].iloc[j+i]).addNext(self.user['contentId'].iloc[j-1]) 
        
                    
    def getTopK(self,state,k):
        #get most likely prob for state(given as array of ids)
        #implement smoothing, weighting probibilties 
        tally = Counter({})
        current = self.tree
        for item in state:
            nxt = current.subSearch(item)
            #weights probablities of next with min-max regularization
            A = {}
            for key in nxt.next.keys():
                
                A[key] = nxt.next[key] * ((nxt.support)/(self.max))
            #A = nxt.next.update()
            tally.update(A)
            
            if nxt == None:
                return current
            current = nxt
        return tally.most_common(k)
        
        
    
    #def findProb(self,):
    
        
    
mm = MTMarkovModel(user,3)
mm.buildModel()
print("Max: ", mm.max)
print("Mean: ", mm.mean)
#for i in range (0,100):
    #print(mm.tree.children[i].id)7f32680caeb8
#print("Test: ", mm.tree.search(pd.Series([-6623581327558800021,4876769046116846438,1005751836898964351])).id)
topk = mm.getTopK([-6623581327558800021,4876769046116846438,1005751836898964351],3)
print(topk)
